In [1]:
# read image data
import cv2
def read_image(path):
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    return img

In [2]:
import json
def get_die_data(path):
    data = ""
    with open(path) as f:
        data = json.load(f)
    die_data,care_areas,exclusion_zones = data['die'],data['care_areas'],data['exclusion_zones']
    return die_data,care_areas,exclusion_zones

In [3]:
die_data,care_areas,exclusion_zones = get_die_data("/Users/shivshankar/Shiv/KLA_Hackathon/Level_3_data/input.json")
single_die = ""

In [4]:
frame_height = ''
frame_width = ''
img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_3_data/wafer_image_3.png")
(frame_height,frame_width) = img.shape
# print(frame_height,frame_width)
number_of_h_frame,number_of_v_frame = (die_data['width']*die_data['columns'])//frame_width,(die_data['height']*die_data['rows'])//frame_height

In [5]:
# print(number_of_h_frame,number_of_v_frame)

In [6]:
# check if pixel in care_area
def check_care_area_pixel(i,j,value):
    maxx_care = value['bottom_right']['x']
    miny_care = value['bottom_right']['y']
    minx_care = value['top_left']['x']
    maxy_care = value['top_left']['y']
    valid = 0
    if i >= minx_care and i <= maxx_care and j >= miny_care and j <= maxy_care:
        valid = 1
    return valid

In [7]:
# check if pixel in exclusion zones
def check_exclusion_zones_pixel(i,j,value):
    maxx_care = value['bottom_right']['x']
    miny_care = value['bottom_right']['y']
    minx_care = value['top_left']['x']
    maxy_care = value['top_left']['y']
    valid = 0
    if i >= minx_care and i <= maxx_care and j >= miny_care and j <= maxy_care:
        valid = 1
    return valid

In [8]:
# check weather count this error
def check_index(i,j,care_areas,exclusion_zones):
    is_in_care_area = 0
    is_in_exclusion_zones = 0
    for value in care_areas:
        is_in_care_area = is_in_care_area or check_care_area_pixel(i,j,value)
    for value in exclusion_zones:
        is_in_exclusion_zones = is_in_exclusion_zones or check_exclusion_zones_pixel(i,j,value)
    if is_in_exclusion_zones:
        return 0
    if is_in_care_area:
        return 1
    return 0

In [9]:
# combine image row wise
def combine_image_row_wise(i,j,dirn):
    first_val = 1
    c = ""
    for ind in range(i,j):
        img = read_image("/Users/shivshankar/Shiv/KLA_Hackathon/Level_3_data/wafer_image_"+str(ind)+".png")
        if first_val == 1:
            c = img
            first_val = 0
        else:
            if dirn == 1:
                c = np.concatenate((img,c),axis=1)
            else:
                c = np.concatenate((c,img),axis=1)
    return c

In [10]:
# combine_image_row_wise(1,6)
# ind = 1
# for i in range(1,4):
#     c_value = combine_image_row_wise(ind,ind+5)
#     ind+=5
# a = [[1,2,3],[3,4,5]]
# b = [[1,2,3],[3,4,5]]
# c = np.concatenate((a,b),axis=0)
# print(c)

In [11]:
# combine all image
import numpy as np
def combine_image():
    first_val = 1
    c = ""
    dirn = 0
    ind = 1
#     number_of_h_frame,number_of_v_frame
    for i in range(1,number_of_v_frame+1):
        c_value = combine_image_row_wise(ind,ind+number_of_h_frame,dirn)
        ind+=number_of_h_frame
        if first_val == 1:
            c = c_value
            first_val = 0
        else:
            c = np.concatenate((c,c_value),axis=0)
        dirn = 1-dirn
    print(c.shape)
    return c

In [12]:
# get all individual dies
import numpy as np
def individual_dies():
    c = combine_image()
    (j,i) = c.shape
    dies = dict()
    dies_count = 1
    for indj in range(0,j,die_data['height']):
        for ind in range(0,i,die_data['width']):
            a = c[indj:indj+die_data['height'],ind:ind+die_data['width']]
            dies[str(dies_count)] = a
            dies_count+=1
    print("dies_count %s " %(dies_count))
    return dies

In [13]:
all_dies = individual_dies()
# from PIL import Image as im
# data = im.fromarray(all_dies['1'])
# data.save('gfg_dummy_pic.png')

(7500, 16000)
dies_count 41 


In [14]:
# return optimal die

#calculate the frequency of pixel value at each coordinates
def calculate_freq():
    freq = []
    for i in range(die_data['height']):
        a = []
        for j in range(die_data['width']):
            s = dict()
            a.append(s)
        freq.append(a)

    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if str(value[i][j]) in freq[i][j]:
                    freq[i][j][str(value[i][j])]+=1
                else:
                    freq[i][j][str(value[i][j])]=1

    return freq

In [15]:
# return optimal die
def get_optimal_die():
    freq = calculate_freq()
    optimal_die = np.zeros((die_data['height'],die_data['width']))
    for i in range(die_data['height']):
        for j in range(die_data['width']):
            max_key = max(freq[i][j],key=freq[i][j].get)
            optimal_die[i][j] = max_key
            
    return optimal_die

In [16]:
# calculate error
# given all dies and optimal dies, calculate the error
def calculate_errors():
    optimal_die = get_optimal_die()
#     all_dies = individual_dies()
    errors = []
    for [key,value] in all_dies.items():
        for i in range(die_data['height']):
            for j in range(die_data['width']):
                if value[i][j] != optimal_die[i][j] and check_index(j,die_data['height']-1-i,care_areas,exclusion_zones) == 1:
                    error = []
                    error.append(key)
                    error.append(j)
                    error.append(die_data['height']-1-i)
                    errors.append(error)
    return errors

In [17]:
# write file to csv
import csv
def make_csv_file():
    errors = calculate_errors()
    print(len(errors))
    with open('answer.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(errors)

In [18]:
make_csv_file()

126347
